In [3]:
#movie emoji HW
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

llm = ChatOpenAI(
  temperature=0.1,
)

examples = [
    {
        "question": "영화를 나타내는 세 개의 이모티콘은 무엇인가요?",
        "answer": """
        영화를 나타내는 세 개의 이모티콘은 다음과 같습니다.:
        영화이름: 탑건
        영화 상징 아이콘: 🛩️👨‍✈️🔥
        """,
    },
    {
        "question": "영화를 나타내는 세 개의 이모티콘은 무엇인가요?",
        "answer": """
        영화를 나타내는 세 개의 이모티콘은 다음과 같습니다.:
        영화이름: 대부
        영화 상징 아이콘: 👨‍👨‍👦🔫🍝
        """,
    },
    {
        "question": "영화를 나타내는 세 개의 이모티콘은 무엇인가요?",
        "answer": """
        영화를 나타내는 세 개의 이모티콘은 다음과 같습니다.:
        영화이름: 엘리멘탈
        영화 상징 아이콘: 🫗🔥❤️
        """,
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  suffix="Human: 영화{movie} 를 나타내는 세 개의 이모티콘은 무엇인가요?",
  input_variables=["movie"]
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 영화의 특징을 함축적인 아이콘으로 표현할 줄 아는 센스있는 영화 평론가 입니다."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["chat_history"]

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm 

def invoke_chain(question):
    result = chain.invoke({"question":question})
    memory.save_context({"input":question},{"output":result.content})
    print("result-->",result)

invoke_chain("내 이름은 혜영이야")


{'question': '내 이름은 혜영이야'}
result--> content='안녕하세요, 혜영님! 영화의 특징을 아이콘으로 표현하면, 마치 영화 포스터처럼 감각적이고 독창적인 분석을 제공할 것으로 예상됩니다. 영화에 대한 열정과 통찰력이 돋보이는 혜영님의 평론을 읽는 것은 항상 즐거운 시간이 될 것입니다. 함께 영화 이야기를 나누어보는 것을 기대합니다!'


In [4]:
invoke_chain("나는 영화 '파묘'를 좋아해")

{'question': "나는 영화 '파묘'를 좋아해"}
result--> content="'파묘'를 아이콘적인 아이콘으로 표현하자면, 검은 고양이의 눈과 빨간 립스틱이 그려진 레트로한 스타일의 포스터일 것입니다. 이 영화는 신비로운 분위기와 미스터리한 요소가 어우러진 작품으로, 관객들을 마법 같은 세계로 이끌어가는 매력이 있습니다. '파묘'는 그 어떤 다른 영화와도 구별되는 독특한 분위기와 스타일을 자랑하는 작품이죠."


In [5]:
invoke_chain("영화 파묘를 3개의 이모티콘으로 나타낸다면 어떤 이모티콘으로 나타낼 수 있을까")

{'question': '영화 파묘를 3개의 이모티콘으로 나타낸다면 어떤 이모티콘으로 나타낼 수 있을까'}
result--> content='🐱💄🔮'


In [6]:
invoke_chain("영화 에일리언을 3개의 이모티콘으로 나타낸다면 어떤 이모티콘으로 나타낼 수 있을까")

{'question': '영화 에일리언을 3개의 이모티콘으로 나타낸다면 어떤 이모티콘으로 나타낼 수 있을까'}
result--> content='👽🚀🔥'


In [7]:
invoke_chain("내 이름이 뭘까?")

{'question': '내 이름이 뭘까?'}
result--> content='당신의 이름은 혜영입니다.'


In [8]:
invoke_chain("나는 영화 엘리멘탈도 참 좋아해.")

{'question': '나는 영화 엘리멘탈도 참 좋아해.'}
result--> content="'엘리멘탈'은 마법적인 요소와 자연의 힘을 다루는 영화로, 아이콘적인 이모티콘으로 표현하자면 다음과 같을 것 같아요: 🌿🔥💧"


In [9]:
invoke_chain("내가 좋아하는 영화들을 모두 아이콘으로 나타내 줘.")

{'question': '내가 좋아하는 영화들을 모두 아이콘으로 나타내 줘.'}
result--> content='파묘: 🐱💄🔮\n에일리언: 👽🚀🔥\n엘리멘탈: 🌿🔥💧'
